# Medium scraper
Inspired by https://github.com/lazuxd/medium-scraping/blob/master/medium_scraping.ipynb 

In [12]:
tag = 'white-supremacy'
year = str(2020) # 2015 gives only a few articles
month = f'{6:02}'
url = f'https://medium.com/tag/{tag}/archive/{year}/{month}'
print(url)

https://medium.com/tag/white-supremacy/archive/2020/06


In [4]:
import requests
from bs4 import BeautifulSoup

response = requests.get(url, allow_redirects=True)
soup = BeautifulSoup(response.content)
articles = soup.find_all("div", class_="postArticle postArticle--short js-postArticle js-trackPostPresentation js-trackPostScrolls")

In [6]:
len(articles)

2

In [ ]:
for year in range(2010, 2022):

## Get URLs of articles
Have to first search by month, then if there are more than 10 articles listed, try days

In [7]:
article_urls = [article.find_all("a")[3]['href'].split('?')[0] for article in articles]
article_urls

['https://medium.com/news-and-politics/to-my-friends-who-have-not-posted-acknowledged-shared-or-commented-about-the-tragedy-in-a00798464010',
 'https://medium.com/@enajlaura/thoughts-on-the-charleston-shooting-or-white-supremacy-continues-to-be-awful-pervasive-8ffba481a8f2']

In [8]:
# Get article text
response = requests.get(article_urls[0])
response

<Response [200]>

In [9]:
response.content

b'<!doctype html><html lang="en"><head><title data-rh="true">To My Friends Who Have Not Posted, Acknowledged, Shared or Commented About the Tragedy in Charleston | by Susan Ito | News Report: News, Current Events, Politics, etc | Medium</title><meta data-rh="true" charset="utf-8"/><meta data-rh="true" name="viewport" content="width=device-width,minimum-scale=1,initial-scale=1,maximum-scale=1"/><meta data-rh="true" name="theme-color" content="#000000"/><meta data-rh="true" name="twitter:app:name:iphone" content="Medium"/><meta data-rh="true" name="twitter:app:id:iphone" content="828256236"/><meta data-rh="true" property="al:ios:app_name" content="Medium"/><meta data-rh="true" property="al:ios:app_store_id" content="828256236"/><meta data-rh="true" property="al:android:package" content="com.medium.reader"/><meta data-rh="true" property="fb:app_id" content="542599432471018"/><meta data-rh="true" property="og:site_name" content="Medium"/><meta data-rh="true" property="og:type" content="art

In [11]:
soup = BeautifulSoup(response.content)
paras = [p.text for p in soup.find_all('p', class_='pw-post-body-paragraph')]
paras

['[Edit: June 20th — I wrote this very soon after the shootings occured. I was in a raw state of despair. Since then, I have realized that these words, although passionately felt, have blamed and made assumptions about others. In retrospect, I think that what I was feeling was alone in my grief and outrage and despair. I wanted to feel less alone. I wanted to know who my people were. I wanted to know who was feeling what I felt. Rather than just delete this posting, I’m going to leave it stand, but with the acknowledgment that one can never know what others are truly thinking, feeling or doing, nor do I have the right to dictate what others post on their social media. I can only hope that others who share my feelings will reach out, somehow.]',
 'It’s been twenty-four hours now since it happened. Since a white supremacist terrorist entered African Wesleyan Methodist Episcopal Church in Charleston, South Carolina, and shot to death 9 people in a prayer meeting.',
 'I was driving home la

In [14]:
# Find title element
for el in soup(text='To My Friends Who Have Not Posted, Acknowledged, Shared or Commented About the Tragedy in Charleston'):
    print(el.parent)

<h1 class="pw-post-title jc jd je bm jf jg jh ji jj jk jl jm jn jo jp jq jr js jt ju jv jw jx jy jz ka fw" id="e32b">To My Friends Who Have Not Posted, Acknowledged, Shared or Commented About the Tragedy in Charleston</h1>


In [16]:
title = soup.find('h1', class_='pw-post-title').text
title

'To My Friends Who Have Not Posted, Acknowledged, Shared or Commented About the Tragedy in Charleston'